Este notebook tiene el compromiso de categorizar los eventos, data cleaning y  determinar categorías del archivo ny_events.csv el cual contiene informacion historica de eventos (deportivos, cultitales, etc..) para la ciudad de nueva york. 
Adema se diseña un script (Programa) que lee la descripción del evento y lo categoriza.

In [1]:
#Se importa pandas como libreria de procesamiento de datos
import pandas as pd

In [2]:
# Leemos el archivo ny_events.csv tal como viene de la pagina 
# https://data.ny.gov/Transportation/511-NY-Sporting-Concert-and-Special-Events-Beginni/3ha4-4nfg
df=pd.read_csv("C:\\Users\\jisaza\\Desktop\\DS4A_workspace\\datathon\\data\\ny_events.csv")

# Convertimos lowercase (minuscula) los string presentes en las columnas "Event Type" y "Event Description" 
df["Event Type"]=df["Event Type"].str.lower()
df["Event Description"]=df["Event Description"].str.lower()


In [3]:
# Se aplica un split en la fecha del evento para extraer el string "PM" para poder
# convertir las horas de un formato de 12 horas a uno de 24 horas (militar)

# creo un dataframe new con tres columnas que me permiten crear una cuarta columna
# llamada "event timestamp" en foramto de hora militar usando un condicional de PM y 
# introduciendo un timedelta de 12 horas
new = df["Create Time"].str.split(" ", n = 2, expand = True)
new.columns=["event date","start time","afternoon"]
new["event timestamp"]=new["event date"].astype(str)+" "+new["start time"].astype(str)
new["event timestamp"]=pd.to_datetime(new["event timestamp"])
new["event timestamp"][new["afternoon"]=="PM"]=new["event timestamp"][new["afternoon"]=="PM"]+pd.Timedelta(hours=12)





C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
# Introduzco la columna event date a partir del dataframe "new" 
# en nuestro dataframe original
df["event date"]=new["event date"]
df["event start timestamp"]=new["event timestamp"]
# Filtro por eventos entre 2014 y 2015, esta condicion viene determinada por
# la disponibilidad de informacion de green trips y yellow trips
start_date = '2013-12-31'
end_date = '2016-01-01'
#Aplico el filtro de fechas
mask = (df['event start timestamp'] > start_date) & (df['event start timestamp'] < end_date)
df=df[mask]


In [6]:
# Usando la columna "Event Description" extraigo eventos de la categoria especial event a las categorias 
# mas recurrentes {concert,hockey game,basketball game,football game,soccer game,show,baseball game
#                  wrestling }

df["Event Type"][df['Event Description'].str.contains("concert", case=False)]="concert"
df["Event Type"][df['Event Description'].str.contains("hockey", case=False)]="hockey game"
df["Event Type"][df['Event Description'].str.contains("basketball", case=False)]="basketball game"
df["Event Type"][df['Event Description'].str.contains("football", case=False)]="football game"
df["Event Type"][df['Event Description'].str.contains("soccer", case=False)]="soccer game"
df["Event Type"][df['Event Description'].str.contains("show", case=False)]="show"
df["Event Type"][df['Event Description'].str.contains("yankees", case=False)]="baseball game"
df["Event Type"][df['Event Description'].str.contains("yankees:", case=False)]="baseball game"
df["Event Type"][df['Event Description'].str.contains("mets:", case=False)]="baseball game"
df["Event Type"][df['Event Description'].str.contains("mets", case=False)]="baseball game"
df["Event Type"][df['Event Description'].str.contains("baseball", case=False)]="baseball game"
df["Event Type"][df['Event Description'].str.contains("wrestling", case=False)]="wrestling"


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWa

In [7]:
#corpus = df["Event Description"].str.lower().str.split(" ", n = 4, expand = True)
#c1=corpus.iloc[:,1]=="special"  
#c2=corpus.iloc[:,0]=="special"
#corpus[c1 | c2]
#corpus
#corpus.iloc[:,1].value_counts()

In [12]:
#Conservo las columnas relevantes pora el analisis
df=df[['Event Type','event date','event start timestamp','Latitude','Longitude']]
#df.head()

,Event Type,event date,event start timestamp,Latitude,Longitude
4008,concert,12/31/2015,2015-12-31 08:03:00,40.750320,-73.991172
4009,special event,12/31/2015,2015-12-31 02:27:00,40.759902,-73.980393
4010,christmas spectacular,12/31/2015,2015-12-31 01:30:00,40.759902,-73.980393
4011,special event,12/30/2015,2015-12-30 10:56:00,40.759902,-73.980393
4012,special event,12/30/2015,2015-12-30 10:14:00,40.759902,-73.980393


In [14]:
df.to_csv("data\\ny_events_clean.csv")